In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans

In [29]:
# Read data
dataset_BJ = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Beijing_labeled.csv')
dataset_SY = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Shenyang_labeled.csv')

dataset_SH = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Shanghai_labeled.csv')
dataset_GZ = pd.read_csv('/Users/liruohan/Documents/DocumentsMacBookPro/2024Spring/StudyPeriod01/DesignOfAISystems/Assignment/module3/Cities/Guangzhou_labeled.csv')


#Train dataset without the PM labels
dataset_BJ_SY = pd.concat([dataset_BJ.iloc[:,:], dataset_SY.iloc[:,:]], ignore_index=True)


In [30]:
class kmeans_classifier:
    def __init__(self, n_clusters=2):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        # The kmeans clustering result labels
        self.labels = {}

    def Fit(self, X, Y):
        self.kmeans.fit(X)
        self.clusters = self.kmeans.predict(X)
        for cluster_id in range(self.n_clusters):
            #Get labels from Y
            indices = np.where(self.clusters == cluster_id)[0]
            Y_labels = Y[indices]

            label_1_size = Y_labels.sum()
            label_0_size = Y_labels.shape[0] - label_1_size
            if(label_0_size > label_1_size):
                self.labels[cluster_id] = 0
            else:
                self.labels[cluster_id] = 1

    def Predict(self, X):
        clusters = self.kmeans.predict(X)
        Y = np.array([self.labels[cluster] for cluster in clusters])
        return Y
    
    def Score(self, Y_predicted, Y):
        accuracy = np.mean(Y_predicted == Y)
        return accuracy


In [31]:
#Cross validation with c=6
size = dataset_BJ_SY.shape[0]
c = 6
validationSize = int(size/c)
#initialize k
k_0 = 0
# Choose k
accuracy_train = np.zeros(11)
accuracy_validate = np.zeros(11)

for k in range(2,11):
    for i in range(6):
        if (i < 5):
            dataset_validate = dataset_BJ_SY.iloc[i*validationSize : (i+1)*validationSize]
            dataset_train = pd.concat([dataset_BJ_SY.iloc[ : i*validationSize], dataset_BJ_SY.iloc[(i+1)*validationSize : ]], ignore_index=True)
        else:
            dataset_validate = dataset_BJ_SY.iloc[i*validationSize : ]
            dataset_train = pd.concat([dataset_BJ_SY.iloc[ : i*validationSize], dataset_BJ_SY.iloc[(i+1)*validationSize : ]], ignore_index=True)

        #features and labels
        X_train = dataset_train.iloc[:, :-1]
        Y_train = dataset_train.iloc[:, -1]
        X_validate = dataset_validate.iloc[:, :-1]
        Y_validate = dataset_validate.iloc[:, -1]

        #Initialize
        kmeansClassifier = kmeans_classifier(k)
        #Fit
        kmeansClassifier.Fit(X_train,Y_train)
        #Validation
        Y_train_predicted = kmeansClassifier.Predict(X_train)
        Y_validate_predicted = kmeansClassifier.Predict(X_validate)
        #Evaluate
        accuracy_validate[k] += kmeansClassifier.Score(Y_train_predicted,Y_train)
        accuracy_train[k] += kmeansClassifier.Score(Y_validate_predicted,Y_validate)

    accuracy_validate[k] = accuracy_validate[k] / c
    accuracy_train[k] = accuracy_train[k] / c

print("Training accuracy: ")
print(accuracy_train[2:])
print("Validation accuracy: ")
print(accuracy_validate[2:])

for k in range(11):
    if accuracy_validate[k] == accuracy_validate.max():
        #k_0 is the k value adopted for kmeans clustering
        k_0 = k
        print(k)


Training accuracy: 
[0.72494831 0.72494831 0.73220972 0.7308266  0.7308266  0.71086966
 0.75504413 0.74812422 0.75054469]
Validation accuracy: 
[0.72489294 0.72489294 0.72965879 0.729866   0.73532256 0.74519961
 0.75873739 0.75086338 0.74699544]
8


In [32]:
#Initialize
kmeansClassifierT = kmeans_classifier(k_0)
#Fit with k = k_0 with all the data in BJ and SY
X_train = dataset_BJ_SY.iloc[:, :-1]
Y_train = dataset_BJ_SY.iloc[:, -1]
kmeansClassifierT.Fit(X_train, Y_train)

#Predict for data in Beijing, Shenyang, Guangzhou and Shanghai
X_BJ = dataset_BJ.iloc[:, :-1]
Y_BJ = dataset_BJ.iloc[:, -1]
accuracy_BJ = kmeansClassifierT.Score(kmeansClassifierT.Predict(X_BJ), Y_BJ)
print(f"The accuracy of predicted PM_HIGH for dataset Beijing is:  {accuracy_BJ:.4f}")

X_SY = dataset_SY.iloc[:, :-1]
Y_SY = dataset_SY.iloc[:, -1]
accuracy_SY = kmeansClassifierT.Score(kmeansClassifierT.Predict(X_SY), Y_SY)
print(f"The accuracy of predicted PM_HIGH for dataset Shenyang is:  {accuracy_SY:.4f}")

X_GZ = dataset_GZ.iloc[:, :-1]
Y_GZ = dataset_GZ.iloc[:, -1]
accuracy_GZ = kmeansClassifierT.Score(kmeansClassifierT.Predict(X_GZ), Y_GZ)
print(f"The accuracy of predicted PM_HIGH for dataset Guangzhou is:  {accuracy_GZ:.4f}")

X_SH = dataset_SH.iloc[:, :-1]
Y_SH = dataset_SH.iloc[:, -1]
accuracy_SH = kmeansClassifierT.Score(kmeansClassifierT.Predict(X_SH), Y_SH)
print(f"The accuracy of predicted PM_HIGH for dataset Shanghai is:  {accuracy_SH:.4f}")


The accuracy of predicted PM_HIGH for dataset Beijing is:  0.7339
The accuracy of predicted PM_HIGH for dataset Shenyang is:  0.8204
The accuracy of predicted PM_HIGH for dataset Guangzhou is:  0.9223
The accuracy of predicted PM_HIGH for dataset Shanghai is:  0.8372
